## Cell Density - using Tensorflow

### Counting number of cells within a given image

A regression problem using image data

Ben Halligan 2020/12/08

- https://arxiv.org/pdf/1803.08450.pdf


In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

import tensorflow.keras as keras

# model using VGG16 to start
from keras.applications import VGG16, InceptionResNetV2
from tensorflow.keras import Input, layers, models

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dropout, Dense, Flatten
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers

from tensorflow.keras.metrics import Precision

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold # might not need
from sklearn.metrics import accuracy_score, precision_score, recall_score

import PIL
import pathlib

from keras.preprocessing.image import ImageDataGenerator
from PIL import Image

data_path = 'data/edges/'

print(tf.config.list_physical_devices('GPU'))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
mirrored_strategy = tf.distribute.MirroredStrategy(['/gpu:0','/gpu:1'])

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
Num GPUs Available:  2
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [27]:
img_size = (200, 200)
batch_size = 16

# datagenerator

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split = 0.2
)

# train
train_gen = datagen.flow_from_directory(
    data_path,
    target_size=img_size,
    color_mode='grayscale',
    batch_size = batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

# test
test_gen = datagen.flow_from_directory(
    data_path,
    target_size=img_size,
    color_mode='grayscale',
    batch_size = batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='validation'
)

Found 312 images belonging to 6 classes.
Found 72 images belonging to 6 classes.


In [28]:
model = models.Sequential()

# Input
inputs = Input(shape=img_size+(1,))

model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization(1))
model.add(Dropout(0.5))

# Flatten
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Fully Connected Layers
model.add(Dense(4, activation='relu'))

# regression end
model.add(Dense(1, activation='linear'))

model.build(new_input.shape)

mse = tf.keras.losses.MeanSquaredError(reduction='sum_over_batch_size', name='mean_squared_error')
model.compile(optimizer='SGD',loss='mean_absolute_error',
             metrics=['accuracy'])
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 200, 200, 32)      320       
_________________________________________________________________
batch_normalization_24 (Batc (None, 200, 200, 32)      800       
_________________________________________________________________
dropout_24 (Dropout)         (None, 200, 200, 32)      0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 1280000)           0         
_________________________________________________________________
dense_36 (Dense)             (None, 16)                20480016  
_________________________________________________________________
batch_normalization_25 (Batc (None, 16)                64        
_________________________________________________________________
dropout_25 (Dropout)         (None, 16)              

In [29]:
epochs = 15
history = model.fit(
    train_gen,
    steps_per_epoch = train_gen.samples//batch_size,
    validation_data = test_gen,
    validation_steps = test_gen.samples//batch_size,
    epochs=epochs
)

Epoch 1/15
19/19 [==============================] - 8s 433ms/step - loss: 0.1698 - accuracy: 0.8333 - val_loss: 0.1683 - val_accuracy: 0.8333
Epoch 2/15
19/19 [==============================] - 8s 411ms/step - loss: 0.1700 - accuracy: 0.8333 - val_loss: 0.1700 - val_accuracy: 0.8333
Epoch 3/15
19/19 [==============================] - 8s 421ms/step - loss: 0.1699 - accuracy: 0.8333 - val_loss: 0.1717 - val_accuracy: 0.8333
Epoch 4/15
19/19 [==============================] - 8s 414ms/step - loss: 0.1702 - accuracy: 0.8333 - val_loss: 0.1678 - val_accuracy: 0.8333
Epoch 5/15
19/19 [==============================] - 8s 413ms/step - loss: 0.1699 - accuracy: 0.8333 - val_loss: 0.1722 - val_accuracy: 0.8333
Epoch 6/15
19/19 [==============================] - 8s 411ms/step - loss: 0.1700 - accuracy: 0.8333 - val_loss: 0.1683 - val_accuracy: 0.8333
Epoch 7/15
19/19 [==============================] - 8s 413ms/step - loss: 0.1699 - accuracy: 0.8333 - val_loss: 0.1700 - val_accuracy: 0.8333
Epoch 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

prec = history.history['precision']
val_prec = history.history['val_precision']


loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, prec, label='Training Precision')
plt.plot(epochs_range, val_prec, label='Validation Precision')
plt.legend(loc='lower right')
plt.title('Training and Validation Precision')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()